In [ ]:
# Define features (X) and labels (y)
X = []
y = []

data = data.astype(str)

for sentence_id, group in data.groupby("sentence_id"):
    sentence_features = []
    sentence_labels = []
    for index, row in group.iterrows():
        token_features = {
            "sentence_id": row["sentence_id"],
            "token_id": row["token_id"],
            "word": row["before"]
        }
        sentence_features.append(token_features)
        sentence_labels.append(row["class"])
    X.append(sentence_features)
    y.append(sentence_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True)

# Train the CRF model
crf.fit(X_train, y_train)

# Make predictions
y_pred = crf.predict(X_test)

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_predictions(y_true, y_pred, classes):
    # Convert labels to binary array format
    mlb = MultiLabelBinarizer(classes=classes)
    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true_bin, y_pred_bin)
    print("Overall Accuracy:", accuracy)

    # Calculate precision, recall, and F1-score for each class
    precision = precision_score(y_true_bin, y_pred_bin, average='micro')
    recall = recall_score(y_true_bin, y_pred_bin, average='micro')
    f1 = f1_score(y_true_bin, y_pred_bin, average='micro')
    print(f"Precision: {precision}, Recall: {recall}, F1-score: {f1}")

# Get unique class labels
classes = list(set(y_test + y_pred))

# Example usage
evaluate_predictions(y_test, y_pred, classes)

TypeError: unhashable type: 'list'